In [1]:
def Stock_contract(symbol, secType='STK', exchange='SMART', currency='USD'):
    ''' custom function to create stock contract '''
    contract = Contract()
    contract.symbol = symbol
    contract.secType = secType
    contract.exchange = exchange
    contract.currency = currency
    contract.primaryExchange = "NASDAQ"
    return contract

def run_loop():
    app.run()

def submit_order(contract, direction, qty=100, ordertype='MKT', transmit=True):
    #Create order
    
    order = Order()
    order.action = direction
    order.totalQuantity = qty
    order.orderType = ordertype
    order.transmit = transmit
    
    #submit order
    app.placeOrder(app.nextorderId, contract, order)
    app.nextorderId += 1

In [9]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *
import pandas
import threading
import time

class IBapi(EWrapper, EClient):
    
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []
        self.contract_details = {}
        
    def error(self, reqId, errorCode, errorString):
        print ('Error: ', reqId, errorCode, ' ', errorString)

    def historicalData(self,reqId, bar):

        print (bar)
        self.data.append([bar.date, bar.close, bar.volume, sym_dict[reqId]])
        
    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)
        
    def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)

    def openOrder(self, orderId, contract, order, orderState):
        print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)
    
    def execDetails(self, reqId, contract, execution):
        print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares)
    
    
    def contractDetails(self, reqId: int, contractDetails):
        self.contract_details[reqId] = contractDetails

    def get_contract_details(self, reqId: int, contract):
        self.contract_details[reqId] = None
        self.reqContractDetails(reqId, contract)
    
    def cancel_Order(self, orderId):
        self.cancelOrder(orderId)
        print('Order'+ " " + f"{orderId}" + " " + 'canceld')

In [5]:
def run_loop():
    app.run()

def makeContract(symbol):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    return contract

In [8]:
# INITIALIZING THE ID OF THE ORDER IN THE IBAPI
app = IBapi()
    
app.connect("127.0.0.1", 7497, 0)
time.sleep(1)
app.nextorderId = 1

api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
time.sleep(3)

Error:  -1 502   Couldn't connect to TWS. Confirm that "Enable ActiveX and Socket EClients" 
is enabled and connection port is the same as "Socket Port" on the 
TWS "Edit->Global Configuration...->API->Settings" menu. Live Trading ports: 
TWS: 7496; IB Gateway: 4001. Simulated Trading ports for new installations 
of version 954.1 or newer:  TWS: 7497; IB Gateway: 4002


In [7]:
submit_order(contract = Stock_contract(symbol="NFLX"), direction="BUY")
time.sleep(1)
#submit_order(contract = Stock_contract(symbol="AAPL"), direction="SELL")

Error:  7 10268   The 'EtradeOnly' order attribute is not supported.
